## 1. Full_Data -> sequntial 변환
- full_data: (웹툰책1권~4권까지 == 애니메이션 기준 1기) - 특정 에피소드 3개 제외한 것

In [21]:
import pandas as pd

input_path = "sl_webtoon_full_data.tsv"
output_path = "sl_webtoon_full_data_sequential.tsv"

df = pd.read_csv(input_path, sep='\t', encoding='utf-8')

df = df.fillna("") #결측치 "" 로 변환

cols = ['에피소드명', '대사', '내적설명', '시스템']

rows = []
for ep, group in df.groupby('에피소드명'):
    for idx, row in group.iterrows():
        for col in ['대사', '내적설명', '시스템']:
            text = row[col].strip()
            if text:  
                rows.append({
                    '에피소드': ep,
                    'scene_text': text,
                    'type': col  
                })

seq_df = pd.DataFrame(rows)

seq_df.to_csv(output_path, sep='\t', index=False, encoding='utf-8')

print("변환, 저장 위치:", output_path)
print(seq_df.head(10))


변환, 저장 위치: sl_webtoon_full_data_sequential.tsv
         에피소드                     scene_text  type
0  1권_1화_이중던전    네, 김상식 아저씨. 신경 써 주셔서 감사합니다.    대사
1  1권_1화_이중던전               내 이름은 성진우. E급 헌터  내적설명
2  1권_1화_이중던전         뭘요 하하... 오늘도 잘 부탁드릴게요.    대사
3  1권_1화_이중던전    헌터협회 소속 중 가장 낮은 계급, 최약의 헌터.  내적설명
4  1권_1화_이중던전  어? 안녕하세요. 주희 씨도 이번 레이드 가시는군요.    대사
5  1권_1화_이중던전   나에게 이런 일이 일어날 줄은...상상도 못 했다.  내적설명
6  1권_1화_이중던전            어쩌다보니 그렇게 됐네요 하하...    대사
7  1권_1화_이중던전                        대한민국 서울  내적설명
8  1권_1화_이중던전         E급 던전이었는데 저만 다쳐서 나왔어요.    대사
9  1권_1화_이중던전                     E급 헌터 성진우.  내적설명


## 2. 한국어 임베딩 모델로 임베딩 생성 + faiss 인덱스+ Rag db 구축

In [24]:
import pandas as pd
import numpy as np
import faiss
import pickle
from sentence_transformers import SentenceTransformer

data_path = "sl_webtoon_full_data_sequential.tsv"

df = pd.read_csv(data_path, sep="\t")
df["text"] = df[["에피소드", "scene_text", "type"]].fillna("").agg(" ".join, axis=1)

model = SentenceTransformer('jhgan/ko-sroberta-multitask') #한국어 임베딩 모델 

embeddings = model.encode(df["text"].tolist(), convert_to_numpy=True)

dim = embeddings.shape[1]  # 모델 출력 차원 자동 확인
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

faiss.write_index(index, "solo_leveling_faiss_ko.index")
with open("solo_leveling_texts.pkl", "wb") as f:
    pickle.dump(df["text"].tolist(), f)

print("RAG DB 구축 완료, 문장 수:", len(df))
#문장 수 : 549


RAG DB 구축 완료, 문장 수: 549


## 3. index, pickle 확인 임베딩 검색

In [26]:
import faiss
import pickle
from sentence_transformers import SentenceTransformer


index = faiss.read_index("solo_leveling_faiss_ko.index")
with open("solo_leveling_texts.pkl", "rb") as f:
    texts = pickle.load(f)

model = SentenceTransformer('jhgan/ko-sroberta-multitask')

query = "성진우는 몇 급 헌터?"

query_vec = model.encode([query])
D, I = index.search(query_vec, k=10)  # 상위 5개

print("유사 장면 Top 10 \n")
for idx, score in zip(I[0], D[0]):
    print(f"[점수: {score:.4f}] {texts[idx]}")
#점수 낮을 수록 유사

유사 장면 Top 10 

[점수: 83.3571] 1권_1화_이중던전 내 이름은 성진우. E급 헌터 내적설명
[점수: 88.2755] 2권_4화_보스전 헌터 성진우입니다. 대사
[점수: 93.2208] 1권_1화_이중던전 E급 헌터 성진우. 내적설명
[점수: 99.7465] 3권_7화_이상한레이드 그 녀석이 원하는 건 실력은 있지만 등급이 낮은 헌터니까. 내적설명
[점수: 102.8061] 3권_7화_이상한레이드 S급 헌터인 황동수를 말하는 겁니까? 대사
[점수: 109.0561] 2권_4화_보스전 능력치를 올릴 수 있는 헌터가 있다? 내적설명
[점수: 109.9106] 1권_1화_이중던전 헌터협회 소속 중 가장 낮은 계급, 최약의 헌터. 내적설명
[점수: 110.5650] 2권_2화_세가지규율 여기서 기다리고 있으면 다른 헌터들이 구조하러 올까요? 대사
[점수: 114.7391] 1권_1화_이중던전 목숨을 거는 위험한 직업 '헌터'. 나라고 좋아서 이 일을 하는 건 아니다. 내적설명
[점수: 119.2824] 1권_1화_이중던전 고졸 출신에 딱히 잘하는 것도 없는 내게, 헌터라는 직업은 피할 수 없는 선택이었다. 내적설명


In [27]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import pickle

# 1. 벡터 DB 로드
embedding = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')
db = FAISS.load_local("solo_leveling_faiss_ko", embedding, allow_dangerous_deserialization=True)  # 🔹 이 옵션 추가


# 2. 예시 질의
query = "성진우는 황동석과 처음 만난 장면에서 무엇을 했어?"
docs = db.similarity_search(query, k=3)

for i, doc in enumerate(docs, 1):
    print(f"[{i}] {doc.page_content}")
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import pickle

# 1. 벡터 DB 로드
embedding = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')
db = FAISS.load_local("solo_leveling_faiss_ko", embedding)

# 2. 예시 질의
query = "성진우는 황동석과 처음 만난 장면에서 무엇을 했어?"
docs = db.similarity_search(query, k=3)

for i, doc in enumerate(docs, 1):
    print(f"[{i}] {doc.page_content}")


[1] [1권_1화_이중던전] #1 내적설명 내 이름은 성진우. E급 헌터
[2] [3권_7화_이상한레이드] #487 대사 S급 헌터인 황동수를 말하는 겁니까?
[3] [1권_3화_퀘스트] #91 대사 주희 씨나 송치열 아저씨는 어떻게 되셨죠?


ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

In [21]:
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
import faiss
import pickle

# 1. 기존 데이터 로드
index_path = "solo_leveling_faiss_ko.index"
texts_path = "solo_leveling_texts.pkl"

index = faiss.read_index(index_path)
with open(texts_path, "rb") as f:
    texts = pickle.load(f)

# 2. 임베딩 모델 준비
embedding = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')

# 3. LangChain 문서화
docs = [Document(page_content=text) for text in texts]
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(docs)})
index_to_docstore_id = {i: str(i) for i in range(len(docs))}

db = FAISS(
    embedding_function=embedding,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# 4. LangChain 호환 구조로 저장
db.save_local("solo_leveling_faiss_langchain")
print("변환 완료: solo_leveling_faiss_langchain 폴더 생성됨")


RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char*) at /project/faiss/faiss/impl/io.cpp:67: Error: 'f' failed: could not open solo_leveling_faiss_ko.index for reading: No such file or directory

## 1. tsv full data load

In [28]:
import pandas as pd


df = pd.read_csv("sl_webtoon_full_data_sequential.tsv", sep="\t")


print(df.head())
print("전체 문장 수:", len(df))
print("컬럼 목록:", df.columns.tolist())

# 549
#에피소드, scene_text, type

         에피소드                     scene_text  type
0  1권_1화_이중던전    네, 김상식 아저씨. 신경 써 주셔서 감사합니다.    대사
1  1권_1화_이중던전               내 이름은 성진우. E급 헌터  내적설명
2  1권_1화_이중던전         뭘요 하하... 오늘도 잘 부탁드릴게요.    대사
3  1권_1화_이중던전    헌터협회 소속 중 가장 낮은 계급, 최약의 헌터.  내적설명
4  1권_1화_이중던전  어? 안녕하세요. 주희 씨도 이번 레이드 가시는군요.    대사
전체 문장 수: 549
컬럼 목록: ['에피소드', 'scene_text', 'type']


In [29]:
import pandas as pd

df = pd.read_csv("sl_webtoon_full_data_sequential.tsv", sep="\t")
print(df.head(3))
print("컬럼:", df.columns.tolist(), "전체 행:", len(df))


         에피소드                   scene_text  type
0  1권_1화_이중던전  네, 김상식 아저씨. 신경 써 주셔서 감사합니다.    대사
1  1권_1화_이중던전             내 이름은 성진우. E급 헌터  내적설명
2  1권_1화_이중던전       뭘요 하하... 오늘도 잘 부탁드릴게요.    대사
컬럼: ['에피소드', 'scene_text', 'type'] 전체 행: 549


In [30]:

df['row_id'] = df.index #인덱스 컬럼 추가 <- 원본 추적용

df['text'] = df.apply(
    lambda x: f"[{x['에피소드']}] #{x['row_id']} {x['type']} {x['scene_text']}", #rag 문장 컬럼 생성
    axis=1
)

print(df['text'].head(3).tolist())


['[1권_1화_이중던전] #0 대사 네, 김상식 아저씨. 신경 써 주셔서 감사합니다.', '[1권_1화_이중던전] #1 내적설명 내 이름은 성진우. E급 헌터', '[1권_1화_이중던전] #2 대사 뭘요 하하... 오늘도 잘 부탁드릴게요.']


In [32]:
texts = df['text'].tolist()
print("최종 문장 수:", len(texts))
# 549

최종 문장 수: 549


## 2. Rag 문장 생성

In [41]:
# 2단계: 최종 RAG 문장 생성
df['row_id'] = df.index  # 원본 추적용 인덱스
df['text'] = df.apply(
    lambda x: f"[{x['에피소드']}] #{x['row_id']} {x['type']} {x['scene_text']}",
    axis=1
)

print("예시 5개:")
for t in df['text'].head(5).tolist():
    print("-", t)

texts = df['text'].tolist()
print("\n최종 문장 수:", len(texts))
#549

예시 5개:
- [1권_1화_이중던전] #0 대사 네, 김상식 아저씨. 신경 써 주셔서 감사합니다.
- [1권_1화_이중던전] #1 내적설명 내 이름은 성진우. E급 헌터
- [1권_1화_이중던전] #2 대사 뭘요 하하... 오늘도 잘 부탁드릴게요.
- [1권_1화_이중던전] #3 내적설명 헌터협회 소속 중 가장 낮은 계급, 최약의 헌터.
- [1권_1화_이중던전] #4 대사 어? 안녕하세요. 주희 씨도 이번 레이드 가시는군요.

최종 문장 수: 549


## 3. 한국어 임베딩 모델 로드, 벡터 db - solo_leveling_faiss_ko



In [36]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')

db = FAISS.from_texts(texts, embedding_model)
print(" 벡터DB 생성 완료. 총 문장 수:", len(texts))

db.save_local("solo_leveling_faiss_ko")
print(" 'solo_leveling_faiss_ko' 폴더에 저장")


 벡터DB 생성 완료. 총 문장 수: 549
 'solo_leveling_faiss_ko' 폴더에 저장


In [37]:
db = FAISS.load_local("solo_leveling_faiss_ko", embedding_model, allow_dangerous_deserialization=True)

#예시
query = "마나석이 뭐지?"
docs = db.similarity_search(query, k=5)

for i, doc in enumerate(docs, 1):
    print(f"[{i}] {doc.page_content}")


[1] [2권_3화_퀘스트 ] #332 대사 던전에 사는 마수라면 마정석을 갖고 있을 줄 알았는데...완전히 다른 부류인가.
[2] [1권_3화_퀘스트] #132 대사 여... 여긴?!  사막...!!
[3] [2권_3화_퀘스트 ] #331 대사 이 녀석들은 마정석 같은 건 안 주나?
[4] [2권_4화_보스전] #457 내적설명 경험이 많으면 많을수록 랭크가 높으면 높을수록 마수들에게서 나오는 마정석은 가치를 더해 간다.
[5] [2권_4화_보스전] #449 대사 문제는 지능인데... 마법과 관련된 스탯일 것 같긴 한데, 이게 피룡할까?


In [39]:
## rag 확인

In [2]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

embedding_model = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')
vectorstore = FAISS.load_local("solo_leveling_faiss_ko", embedding_model, allow_dangerous_deserialization=True)

model_name = "kakaocorp/kanana-nano-2.1b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")

llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="다음 문맥을 참고하여 질문에 답하세요.\n\n문맥:\n{context}\n\n질문:\n{question}\n\n답변:"
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": custom_prompt  }
)

#질문
query = "성진우는 몇 급 헌터지?"
result = qa_chain({"query": query})

print("답변:", result["result"])
print("\n참조 문서:")
for doc in result["source_documents"]:
    print(doc.page_content)


/tmp/ipykernel_1139536/3834059051.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')
Device set to use cuda:0
/tmp/ipykernel_1139536/3834059051.py:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeli

답변: 다음 문맥을 참고하여 질문에 답하세요.

문맥:
[1권_1화_이중던전] #1 내적설명 내 이름은 성진우. E급 헌터

[2권_4화_보스전] #451 대사 헌터 성진우입니다.

[1권_1화_이중던전] #9 내적설명 E급 헌터 성진우.

[3권_7화_이상한레이드] #484 내적설명 그 녀석이 원하는 건 실력은 있지만 등급이 낮은 헌터니까.

[2권_4화_보스전] #468 내적설명 능력치를 올릴 수 있는 헌터가 있다?

질문:
성진우는 몇 급 헌터지?

답변: 성진우는 E급 헌터입니다.  #1 내적설명 내 이름은 성진우. E급 헌터  #9 내적설명 E급 헌터 성진우.  #468 내적설명 능력치를 올릴 수 있는 헌터가 있다? 이 문맥을 보면 성진우는 E급 헌터입니다.  따라서 답은 E급 헌터입니다.  성진우는 몇 급 헌터지?  E급 헌터입니다.  즉, E급입니다.  그래서 답은 E급 헌터입니다.  #468 내적설명 능력치를 올릴 수 있는 헌터가 있다? 이 문맥을 보면 성진우는 E급 헌터입니다.  따라서 답은 E급 헌터입니다.  성진우는 몇 급 헌터지?  E급 헌터입니다.  즉, E급입니다.  그래서 답은 E급 헌터입니다.  #468 내적설명 능력치를 올릴 수 있는 헌터가 있다? 이 문맥을 보면 성진우는 E급 헌터입니다.  따라

참조 문서:
[1권_1화_이중던전] #1 내적설명 내 이름은 성진우. E급 헌터
[2권_4화_보스전] #451 대사 헌터 성진우입니다.
[1권_1화_이중던전] #9 내적설명 E급 헌터 성진우.
[3권_7화_이상한레이드] #484 내적설명 그 녀석이 원하는 건 실력은 있지만 등급이 낮은 헌터니까.
[2권_4화_보스전] #468 내적설명 능력치를 올릴 수 있는 헌터가 있다?


## 4. 황동석 에피소드 

In [3]:
choices = [
    "1-1. 황동석 무리를 모두 처치한다.",
    "1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.",
    "2. 적을 무력화하거나 기절시키고 살려둔다.",
    "3-1. 마정석을 들고 도망친다.",
    "3-2. 시스템을 거부하고 그냥 도망친다."
]

print("\n[선택지]")
for idx, choice in enumerate(choices, start=1):
    print(f"{idx}. {choice}")

user_idx = int(input("\n선택 번호 입력: ")) - 1
user_choice = choices[user_idx]
print(f"\n[사용자 선택]: {user_choice}")

result = qa_chain({"query": user_choice})

retrieved_context = "\n".join([doc.page_content for doc in result["source_documents"]])
print("\n[검색된 근거 문서 예시]")
print(retrieved_context[:600], "...")  # 길면 일부만 출력

# 4. 성진우 말투 대사 생성
prompt = f"""
당신은 웹툰 '나 혼자만 레벨업'의 성진우입니다.
현재 상황:
{retrieved_context}

사용자 선택: {user_choice}

성진우의 말투로 간결하고 자연스러운 대사를 2~3문장 생성하세요.
"""

response = generator(prompt, 
                     max_new_tokens=120, 
                     do_sample=True, 
                     temperature=0.7,
                     return_full_text=False  # 추가
)[0]["generated_text"]
print("\n[성진우 응답]")
print(response)



[선택지]
1. 1-1. 황동석 무리를 모두 처치한다.
2. 1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.
3. 2. 적을 무력화하거나 기절시키고 살려둔다.
4. 3-1. 마정석을 들고 도망친다.
5. 3-2. 시스템을 거부하고 그냥 도망친다.



선택 번호 입력:  3



[사용자 선택]: 2. 적을 무력화하거나 기절시키고 살려둔다.

[검색된 근거 문서 예시]
[2권_4화_보스전] #437 대사 놈의 방어만 무력화시킬 수 있다면, 할 수 있다!
[2권_4화_보스전] #399 내적설명 동시에 이놈의 방어력을 무력화시켜야 해.
[1권_1화_이중던전] #74 대사 지금 당장에라도 우리를 전멸시킬 수 있을 텐데... 왜 그러지 않는 걸까요?
[1권_1화_이중던전] #53 내적설명 언제나 맨몸. 어차피 어정쩡한 무기를 구매해봤자 금방 파괴될 테고, 상해 버린 몸뚱이 따위, 힐러만 있다면 어지간한 중상이 아닌 한 나을 수 있다.
[1권_1화_이중던전] #75 내적설명 죽일 수 있음에도 죽이지 않는다. ...


NameError: name 'generator' is not defined

In [19]:
print(df.columns.tolist())


['에피소드명', '대사', '내적설명', '시스템']


In [1]:
choices = [
    "1-1. 황동석 무리를 모두 처치한다.",
    "1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.",
    "2. 적을 무력화하거나 기절시키고 살려둔다.",
    "3-1. 마정석을 들고 도망친다.",
    "3-2. 시스템을 거부하고 그냥 도망친다."
]

print("\n[선택지]")
for idx, choice in enumerate(choices, start=1):
    print(f"{idx}. {choice}")

user_idx = int(input("\n선택 번호 입력: ")) - 1
user_choice = choices[user_idx]
print(f"\n[사용자 선택]: {user_choice}")

result = qa_chain({"query": user_choice})

retrieved_context = "\n".join([doc.page_content for doc in result["source_documents"]])
print("\n[검색된 근거 문서 예시]")
print(retrieved_context[:600], "...")  # 길면 일부만 출력

# 4. 성진우 말투 대사 생성
prompt = f"""
당신은 웹툰 '나 혼자만 레벨업'의 성진우입니다.
현재 상황:
{retrieved_context}

사용자 선택: {user_choice}

성진우의 말투로 간결하고 자연스러운 대사를 2~3문장 생성하세요.
"""

response = generator(prompt, max_new_tokens=120, do_sample=True, temperature=0.7)[0]["generated_text"]
print("\n[성진우 응답]")
print(response)



[선택지]
1. 1-1. 황동석 무리를 모두 처치한다.
2. 1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.
3. 2. 적을 무력화하거나 기절시키고 살려둔다.
4. 3-1. 마정석을 들고 도망친다.
5. 3-2. 시스템을 거부하고 그냥 도망친다.



선택 번호 입력:  3



[사용자 선택]: 2. 적을 무력화하거나 기절시키고 살려둔다.


NameError: name 'qa_chain' is not defined

In [4]:
choices = [
    "1-1. 황동석 무리를 모두 처치한다.",
    "1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.",
    "2. 적을 무력화하거나 기절시키고 살려둔다.",
    "3-1. 마정석을 들고 도망친다.",
    "3-2. 시스템을 거부하고 그냥 도망친다."
]

print("\n[선택지]")
for idx, choice in enumerate(choices, start=1):
    print(f"{idx}. {choice}")

user_idx = int(input("\n선택 번호 입력: ")) - 1
user_choice = choices[user_idx]
print(f"\n[사용자 선택]: {user_choice}")

result = qa_chain({"query": user_choice})

retrieved_context = "\n".join([doc.page_content for doc in result["source_documents"]])
print("\n[검색된 근거 문서 예시]")
print(retrieved_context[:600], "...")  # 길면 일부만 출력

# 4. 성진우 말투 대사 생성
prompt = f"""
당신은 웹툰 '나 혼자만 레벨업'의 성진우입니다.
현재 상황:
{retrieved_context}

사용자 선택: {user_choice}

성진우의 말투로 간결하고 자연스러운 대사를 2~3문장 생성하세요.
"""

response = generator(prompt, max_new_tokens=120, do_sample=True, temperature=0.7)[0]["generated_text"]
print("\n[성진우 응답]")
print(response)



[선택지]
1. 1-1. 황동석 무리를 모두 처치한다.
2. 1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.
3. 2. 적을 무력화하거나 기절시키고 살려둔다.
4. 3-1. 마정석을 들고 도망친다.
5. 3-2. 시스템을 거부하고 그냥 도망친다.



선택 번호 입력:  3



[사용자 선택]: 2. 적을 무력화하거나 기절시키고 살려둔다.

[검색된 근거 문서 예시]
[2권_4화_보스전] #437 대사 놈의 방어만 무력화시킬 수 있다면, 할 수 있다!
[2권_4화_보스전] #399 내적설명 동시에 이놈의 방어력을 무력화시켜야 해.
[1권_1화_이중던전] #74 대사 지금 당장에라도 우리를 전멸시킬 수 있을 텐데... 왜 그러지 않는 걸까요?
[1권_1화_이중던전] #53 내적설명 언제나 맨몸. 어차피 어정쩡한 무기를 구매해봤자 금방 파괴될 테고, 상해 버린 몸뚱이 따위, 힐러만 있다면 어지간한 중상이 아닌 한 나을 수 있다.
[1권_1화_이중던전] #75 내적설명 죽일 수 있음에도 죽이지 않는다. ...


NameError: name 'generator' is not defined